In [10]:
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot 
from plotly import tools, io
import plotly.graph_objs as go
import numpy as np
import pandas as pd
import math
from scipy.interpolate import interp1d

init_notebook_mode(connected=True)

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

from numpy.linalg import norm


In [17]:
prefix = '/home/simeon/dvp/lenovo/build/m2p_test/' # set the folder used with CSV logs
filename = 'log_temp.csv'

prefix = '/home/simeon/Documents/lenovo/atw/2020-02-26_without_HW_ATW/' # set the folder used with CSV logs
filename = 'log_C0008.csv'

max_iteration = 60

In [18]:
# load csv data
data_log = pd.read_csv(prefix+filename,header=0, index_col=False)


In [19]:
# show the position data diagram
p=[]
#index = np.arange(0,len(data_log))
index = data_log['frame_id']
               
for c in ['real_x','real_y','expected_x','expected_y']:    
    trace=go.Scatter(
        y= data_log[c],
        x= index,
        name = c
    )
    p.append(trace)

                  
axis_template=dict(
    showgrid=True,  #网格
    zeroline=True,  #是否显示基线,即沿着(0,0)画出x轴和y轴
    nticks=20,
    showline=True,
    #title='X axis',
    #mirror='all',
    zerolinecolor="#FF0000"
)
layout=go.Layout(
    xaxis=axis_template,
    yaxis=axis_template,
    title='original position log'
)

fig=go.Figure(
    data=p,
    layout=layout
)
iplot(fig)
io.write_html(fig, file=prefix+filename.replace('.csv','_')+layout.title.text.replace(' ','_')+'.html')


In [20]:
# show the visual & real position error  in pixel
data_diff_x = np.subtract(data_log['real_x'],data_log['expected_x'])
data_diff_y = np.subtract(data_log['real_y'],data_log['expected_y'])

p=[]
#index = np.arange(0,len(data_log))
index = data_log['frame_id']

p.append(go.Scatter(
    y= data_diff_x,
    x= index,
    name='diff x in pixel'
    
))


p.append(go.Scatter(
    y= data_diff_y,
    x= index,
    name='diff y in pixel'
))


layout.title= 'position error in pixel'
fig=go.Figure(
    data=p,
    layout=layout
)
iplot(fig)
io.write_html(fig, file=prefix+filename.replace('.csv','_')+layout.title.text.replace(' ','_')+'.html')

In [21]:
# calculate the latency by shift the element and calculate the var

expected_x = data_log['expected_x']
expected_y = data_log['expected_y']
real_x = data_log['real_x']
real_y = data_log['real_y']

diff_x_sums=[]
diff_y_sums=[]

len_of_list = len(real_x)-max_iteration

expected_x = expected_x[0:len_of_list]
expected_y = expected_y[0:len_of_list]

p=[]
for it in range(max_iteration):
    shifted_realx = real_x.shift(-it)
    shifted_realy = real_y.shift(-it)
    shifted_realx = shifted_realx[0:len_of_list]
    shifted_realy = shifted_realy[0:len_of_list]
    diffx = np.subtract(shifted_realx,expected_x)
    diffy = np.subtract(shifted_realy,expected_y)
    diff_sum_x = np.average( np.abs(diffx))
    diff_sum_y = np.average( np.abs(diffy))
    diff_x_sums.append(diff_sum_x)
    diff_y_sums.append(diff_sum_y)

index = np.arange(0,len(diff_x_sums))
p.append(go.Scatter(
    y = diff_x_sums,
    x = index,
    name = 'total error X'
))
p.append(go.Scatter(
    y = diff_y_sums,
    x = index,
    name = 'total error Y'
))
layout.xaxis['title'] = 'latency'
layout.yaxis['title'] = 'total position error'
layout.title= 'Position error on latency simulation'
fig=go.Figure(
    data=p,
    layout=layout
)
iplot(fig)
io.write_html(fig, file=prefix+filename.replace('.csv','_')+layout.title.text.replace(' ','_')+'.html')

    
    
    
    
# draw the diagramm after best match    
best_match_latency_x = np.argmin(diff_x_sums)
best_match_latency_y = np.argmin(diff_y_sums)
# best_match_latency = min(best_match_latency_x,best_match_latency_y)
range_of_diffx =abs( max(data_log['real_x'])-min(data_log['real_x']))
range_of_diffy =abs( max(data_log['real_y'])-min(data_log['real_y']))
if (range_of_diffx > range_of_diffy) :
    best_match_latency = best_match_latency_x
else:
    best_match_latency = best_match_latency_y
# temp selection for debug    
#best_match_latency = best_match_latency_x


print("best_match_latency_x = %f" %(best_match_latency_x) )
print("best_match_latency_y = %f" %(best_match_latency_y) )
print("chose best match latency = %f" %(best_match_latency) )

shifted_realx = real_x.shift(-best_match_latency)
shifted_realx = shifted_realx[0:len_of_list]

shifted_realy = real_y.shift(-best_match_latency)
shifted_realy = shifted_realy[0:len_of_list]

p=[]
trace=go.Scatter(
        y= shifted_realx,
        x= np.arange(0,len(shifted_realx)),
        name='shifted real_x'
    
    )
p.append(trace)

trace=go.Scatter(
        y= expected_x,
        x= np.arange(0,len(expected_x)),
        name='expected_x'
    )
p.append(trace)

trace=go.Scatter(
        y= shifted_realy,
        x= np.arange(0,len(shifted_realy)),
        name='shifted real_y'
    
    )
p.append(trace)

trace=go.Scatter(
        y= expected_y,
        x= np.arange(0,len(expected_y)),
        name='expected_y'
    )
p.append(trace)

layout.xaxis['title'] = 'time'
layout.yaxis['title'] = 'position'
layout.title= 'Position after latency match'
fig=go.Figure(
    data=p,
    layout=layout
)
iplot(fig)
io.write_html(fig, file=prefix+filename.replace('.csv','_')+layout.title.text.replace(' ','_')+'.html')






# show the visual & real position error  in pixel
data_diff_x_shifted = np.subtract(shifted_realx,expected_x)
data_diff_x = np.subtract(data_log['real_x'],data_log['expected_x'])


p=[]
index = np.arange(0,len(data_diff_x_shifted)-1)
p.append(go.Scatter(
    y= data_diff_x,
    x= np.arange(0,len(data_diff_x)),
    name='original error x'
    
))
p.append(go.Scatter(
    y= data_diff_x_shifted,
    x= np.arange(0,len(data_diff_x_shifted)-1),
    name='after match errorx'
    
))



layout.title= 'X position error in pixel after latency match'
fig=go.Figure(
    data=p,
    layout=layout
)
iplot(fig)
io.write_html(fig, file=prefix+filename.replace('.csv','_')+layout.title.text.replace(' ','_')+'.html')

data_diff_y_shifted = np.subtract(shifted_realy,expected_y)
data_diff_y = np.subtract(data_log['real_y'],data_log['expected_y'])

p=[]
index = np.arange(0,len(data_diff_y_shifted)-1)
p.append(go.Scatter(
    y= data_diff_y,
    x= np.arange(0,len(data_diff_y)),
    name='original error y'
    
))
p.append(go.Scatter(
    y= data_diff_y_shifted,
    x= np.arange(0,len(data_diff_y_shifted)-1),
    name='after match error y'
    
))



layout.title= 'Y position error in pixel after latency match'
fig=go.Figure(
    data=p,
    layout=layout
)
iplot(fig)
io.write_html(fig, file=prefix+filename.replace('.csv','_')+layout.title.text.replace(' ','_')+'.html')

best_match_latency_x = 4.000000
best_match_latency_y = 13.000000
chose best match latency = 13.000000
